In [40]:
# System libraries:
%load_ext autoreload
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import pandas as pd
import json
import nltk
import numpy as np
import os
import sys
from IPython.core.interactiveshell import InteractiveShell
from pymystem3 import Mystem
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from datetime import datetime
import re
import gensim.downloader as api
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
InteractiveShell.ast_node_interactivity = "all"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
integrum_local_lemmas = pd.read_csv('data/processed/integrum_local_lemmas.csv', converters = {'lemmas_content' : eval, 'lemmas_headline':eval})
integrum_federal_lemmas = pd.read_csv('data/processed/integrum_federal_lemmas.csv', converters = {'lemmas_content' : eval, 'lemmas_headline':eval})

In [6]:
sentences_local = integrum_local_lemmas.lemmas_content
# train word2vec on the two sentences
model_local = gensim.models.Word2Vec(sentences_local, min_count=1)

sentences_federal = integrum_federal_lemmas.lemmas_content
# train word2vec on the two sentences
model_federal = gensim.models.Word2Vec(sentences_federal, min_count=1)

2018-05-29 13:18:55,674 : INFO : collecting all words and their counts
2018-05-29 13:18:55,677 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-05-29 13:18:56,890 : INFO : PROGRESS: at sentence #10000, processed 3849812 words, keeping 95908 word types
2018-05-29 13:18:58,555 : INFO : PROGRESS: at sentence #20000, processed 8794850 words, keeping 138728 word types
2018-05-29 13:19:00,097 : INFO : PROGRESS: at sentence #30000, processed 13515149 words, keeping 177293 word types
2018-05-29 13:19:01,315 : INFO : collected 201515 word types from a corpus of 16945329 raw words and 36017 sentences
2018-05-29 13:19:01,316 : INFO : Loading a fresh vocabulary
2018-05-29 13:19:04,089 : INFO : min_count=1 retains 201515 unique words (100% of original 201515, drops 0)
2018-05-29 13:19:04,090 : INFO : min_count=1 leaves 16945329 word corpus (100% of original 16945329, drops 0)
2018-05-29 13:19:04,740 : INFO : deleting the raw counts dictionary of 201515 items
2018-05-2

2018-05-29 13:20:02,520 : INFO : EPOCH 3 - PROGRESS: at 83.42% examples, 846625 words/s, in_qsize 5, out_qsize 0
2018-05-29 13:20:03,523 : INFO : EPOCH 3 - PROGRESS: at 87.57% examples, 842384 words/s, in_qsize 5, out_qsize 0
2018-05-29 13:20:04,533 : INFO : EPOCH 3 - PROGRESS: at 92.00% examples, 842434 words/s, in_qsize 5, out_qsize 0
2018-05-29 13:20:05,562 : INFO : EPOCH 3 - PROGRESS: at 95.91% examples, 838682 words/s, in_qsize 5, out_qsize 0
2018-05-29 13:20:06,571 : INFO : EPOCH 3 - PROGRESS: at 99.55% examples, 835330 words/s, in_qsize 5, out_qsize 0
2018-05-29 13:20:06,689 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-29 13:20:06,703 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-05-29 13:20:06,708 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-29 13:20:06,709 : INFO : EPOCH - 3 : training on 16945329 raw words (16156975 effective words) took 19.3s, 835195 effective words/s
2018-05-29 13:20:07,72

In [57]:
words_positive = ['нокиа']
number_of_results = 25

print("Integrum local:")
model_local.wv.most_similar(positive=words_positive ,topn=number_of_results)

print("Integrum federal:")
model_federal.wv.most_similar(positive=words_positive ,topn=number_of_results)


Integrum local:


[('элкотека', 0.7536649107933044),
 ('синебрюхофф', 0.7445650696754456),
 ('дженерал', 0.7264657020568848),
 ('валмет', 0.7248556613922119),
 ('икеа', 0.7216954231262207),
 ('моторс', 0.71682208776474),
 ('фортум', 0.7142850756645203),
 ('пит', 0.7071996927261353),
 ('сименс', 0.7054600119590759),
 ('фазер', 0.7002048492431641),
 ('раутаруукка', 0.692550539970398),
 ('текс', 0.689965546131134),
 ('фудлайн', 0.6897444128990173),
 ('Metso', 0.684923529624939),
 ('хонк', 0.6811659336090088),
 ('электролюкс', 0.675460934638977),
 ('альстрем', 0.673250675201416),
 ('валио', 0.6728737354278564),
 ('апит', 0.6716701984405518),
 ('энсо', 0.6696405410766602),
 ('сведвуд', 0.6665652990341187),
 ('ExxonMobil', 0.6660447120666504),
 ('тойота', 0.6650810837745667),
 ('петмол', 0.6631209850311279),
 ('Daimler', 0.6601812839508057)]

Integrum federal:


[('валио', 0.7964282631874084),
 ('скартел', 0.7304878830909729),
 ('сименс', 0.7292211055755615),
 ('фольксваген', 0.7063729763031006),
 ('шкода', 0.6917002201080322),
 ('майкрософт', 0.69150710105896),
 ('оринимать', 0.6872502565383911),
 ('полипласт', 0.681629478931427),
 ('тальго', 0.6795402765274048),
 ('стрим', 0.6783123016357422),
 ('статойло', 0.6762619018554688),
 ('дженерал', 0.6755160093307495),
 ('норск', 0.6744836568832397),
 ('энерго', 0.6739196181297302),
 ('пежо', 0.6733487844467163),
 ('трансконтейнер', 0.6732388734817505),
 ('турпарад', 0.6689261794090271),
 ('нижнекамскшина', 0.6655929088592529),
 ('Valio', 0.6647659540176392),
 ('Teboil', 0.661560595035553),
 ('оверсиз', 0.6607126593589783),
 ('крост', 0.65885329246521),
 ('фортум', 0.6537583470344543),
 ('юниливер', 0.6536508798599243),
 ('акрон', 0.6534441709518433)]

In [66]:
words_to_compare = ('финляндия', 'сша')

print("Integrum local similarity:")
model_local.wv.similarity(*words_to_compare)

print("Integrum federal similarity:")
model_federal.wv.similarity(*words_to_compare)

Integrum local similarity:


0.3859300666593514

Integrum federal similarity:


0.19746752395392744

In [67]:
positive_terms = ['россия','сауна']
negative_terms = ['финляндия']

print("Integrum local similarity:")
model_local.wv.most_similar(positive=positive_terms, negative=negative_terms, topn=5)

print("Integrum federal similarity:")
model_federal.wv.most_similar(positive=positive_terms, negative=negative_terms, topn=5)


Integrum local similarity:


[('кухня', 0.529101550579071),
 ('душевой', 0.5201793909072876),
 ('туалет', 0.49875491857528687),
 ('парная', 0.4677061140537262),
 ('ванна', 0.46202588081359863)]

Integrum federal similarity:


[('баня', 0.5577921867370605),
 ('туалет', 0.5228835344314575),
 ('дорожка', 0.5196024775505066),
 ('одежда', 0.48897117376327515),
 ('развлечение', 0.48832979798316956)]